In [8]:
import pandas as pd
import requests
import datetime
import sqlalchemy
import requests

# Time from  8:00 to 20:00


In [9]:
time_from = "08:00"
time_to = "20:00"
today = datetime.date.today()
tomorrow =today + datetime.timedelta(days=1)
tomorrow= tomorrow.strftime('%Y-%m-%d')
tomorrow
my_icao = ['EDDB', 'EGLL']

In [12]:
my_jsons = []

for icao in my_icao:
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time_from}/{tomorrow}T{time_to}"
    querystring = {
        "withLeg": "true",
        "direction": "Arrival",
        "withCancelled": "false",
        "withCodeshared": "true",
        "withCargo": "false",
        "withPrivate": "false",
        "withLocation": "false"
    }
    
    headers = {
        "X-RapidAPI-Key": "0b123124444444444444444444444444444444444444414",    #your api key here
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    print(response.status_code)
    my_jsons.append(response.json())

200
200


In [ ]:
my_jsons #list of two airports

In [15]:
flights_json = my_jsons[0]['arrivals']

In [16]:
my_jsons[0]['arrivals'][0]['departure']['airport']['icao']

'LSZH'

In [17]:
my_jsons[0]['arrivals'][1]['departure']['airport']['name']

'Frankfurt-am-Main'

In [18]:
my_jsons[0]['arrivals'][1]['arrival']['scheduledTimeLocal']

'2023-09-12 08:55+02:00'

In [19]:
from IPython.display import JSON

In [20]:
JSON(my_jsons)

<IPython.core.display.JSON object>

In [21]:
my_jsons[1]['arrivals'][1]['arrival']['terminal']

'3'

# The important things about the for loop:
```
1. my_jsons = ['one_jason ='Berlin' , 'two_jason = london']this is the big loop(outer for loop)
2.one_json = [flight and we are interested about the arrivals]this is the small loop(inner for loop)
```

<!-- # outer for loop
for element in sequence 
   # inner for loop
   for element in sequence:
       body of inner for loop
   body of outer for loop -->

In [23]:
flights_dict = {'Arrival_airport':[],
                'Departure_airport_icao':[],
                'Departure_airport_city':[],
                'Departure_terminal':[],
                'Arrival_time_local':[] }

for index, one_json in enumerate(my_jsons):
    for anything in one_json.get('arrivals', []):
        flights_dict['Arrival_airport'].append(my_icao[index])

        departure_info = anything.get('departure', {})
        flights_dict['Departure_airport_icao'].append(departure_info.get('airport', {}).get('icao', 'unknown'))
        flights_dict['Departure_airport_city'].append(departure_info.get('airport', {}).get('name', 'unknown'))
        flights_dict['Arrival_time_local'].append(anything.get('arrival', {}).get('scheduledTimeLocal', 'unknown'))

        try:
            flights_dict['Departure_terminal'].append(departure_info['terminal'])
        except KeyError:
            flights_dict['Departure_terminal'].append('unknown')

flights = pd.DataFrame(flights_dict)

# Problem of the Arrival_Time_local

In [24]:
flights['Arrival_time_local'] = flights['Arrival_time_local'].str[:-6]
#exclude the last 6 character of the time utc 2 hr diff

In [25]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2090 entries, 0 to 2089
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Arrival_airport         2090 non-null   object
 1   Departure_airport_icao  2090 non-null   object
 2   Departure_airport_city  2090 non-null   object
 3   Departure_terminal      2090 non-null   object
 4   Arrival_time_local      2090 non-null   object
dtypes: object(5)
memory usage: 81.8+ KB


In [26]:
flights['Arrival_time_local']=pd.to_datetime(flights['Arrival_time_local'])

In [27]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2090 entries, 0 to 2089
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Arrival_airport         2090 non-null   object        
 1   Departure_airport_icao  2090 non-null   object        
 2   Departure_airport_city  2090 non-null   object        
 3   Departure_terminal      2090 non-null   object        
 4   Arrival_time_local      2090 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 81.8+ KB


In [28]:
flights

,Arrival_airport,Departure_airport_icao,Departure_airport_city,Departure_terminal,Arrival_time_local
0,EDDB,LSZH,Zurich,unknown,2023-09-12 08:45:00
1,EDDB,EDDF,Frankfurt-am-Main,1,2023-09-12 08:55:00
2,EDDB,EFHK,Helsinki,2,2023-09-12 08:05:00
3,EDDB,LFPO,Paris,unknown,2023-09-12 08:45:00
4,EDDB,LIML,Milan,unknown,2023-09-12 08:45:00
...,...,...,...,...,...
2085,EGLL,EHAM,Amsterdam,unknown,2023-09-12 19:25:00
2086,EGLL,WSSS,Singapore,3,2023-09-12 19:15:00
2087,EGLL,LEMD,Madrid,4,2023-09-12 19:20:00
2088,EGLL,OOMS,Muscat,unknown,2023-09-12 19:10:00


In [29]:
schema="gans"   # name of the database you want to use here
host="database-gansproject.fhelrtherlthrehhrhrhr.rds.amazonaws.com"        # to connect to your aws server
user="admin"
password="blablablaba" # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [30]:
flights.to_sql('flights',con=con,if_exists='append',index=False)

2090